# Falcon 9 Launch Dashboard
**Author:** Muhammad Munawar Shahzad 

**Date:** August 2025  
**Project:** IBM Applied Data Science Capstone – SpaceX Falcon 9
**Author:** Muhammad Munawar Shahzad 


**Repository:** `falcon9_project`  
## 📌 Overview
This dashboard provides an interactive way to explore Falcon 9 launch data.  
Users can:

- View launch trends by year.
- Explore success rates by launch site.
- Filter data interactively.

## 🎯 Objective of This Notebook
1. Load the cleaned Falcon 9 dataset from `processed` folder.
2. Build a Dash web application with interactive charts.
3. Deploy locally for exploration.

In [9]:
# Step 1: Install necessary libraries (if not already installed)
# !pip install dash plotly pandas --quiet

In [10]:
# Step 2: Import Required Libraries
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load data
df = pd.read_csv("../data/processed/falcon9_cleaned_for_eda.csv")

# Quick check
df.head()

,flight_number,date,booster_version,launch_site,payload,payload_mass_kg,orbit,customer,launch_outcome,booster_landing,launch_year,launch_site_code,success_flag
0,286.0,2024-01-03 03:44:00,F9 B5 B1082‑1,"Vandenberg, SLC‑4E",Starlink: Group 7-9 (22 satellites),16800.0,LEO,SpaceX,Success,Success (OCISLY),2024.0,VS,1
1,286.0,NaN,"Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...",22.0,"Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...","Launch of 22 Starlink v2 mini satellites, incl...",NaN,Lo2Svmsitfstfdcta5k(moaaio5teic,0
2,287.0,2024-01-03 23:04:00,F9 B5 B1076‑10,"Cape Canaveral, SLC‑40",Ovzon-3,1800.0,GTO,Ovzon,Success,Success (LZ‑1),2024.0,CCS,1
3,287.0,NaN,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,26.0,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,Broadband internet provider satellite.[26] Fir...,NaN,BipsFF9ltGwar(lFcswROSAtwdoJ12,0
4,288.0,2024-01-07 22:35:00,F9 B5 B1067‑16,"Cape Canaveral, SLC‑40",Starlink: Group 6-35 (23 satellites),17100.0,LEO,SpaceX,Success,Success (ASOG),2024.0,CCS,1


In [11]:
# Step 3: Initialize the Dash application
app = Dash(__name__)

# Get dropdown options for Launch Site
launch_sites = [{"label": site, "value": site} for site in sorted(df["launch_site"].unique())]
launch_sites.insert(0, {"label": "All Sites", "value": "ALL"})

In [12]:
# Step 4: Define the dashboard layout
app.layout = html.Div([
    html.H1("Falcon 9 Launch Dashboard", style={"textAlign": "center"}),

    # Dropdown for selecting launch site
    dcc.Dropdown(
        id="site-dropdown",
        options=launch_sites,
        value="ALL",
        placeholder="Select a Launch Site",
        style={"width": "50%", "margin": "auto"}
    ),
    
    html.Br(),
    
    # Pie chart for success rates
    dcc.Graph(id="success-pie-chart"),
    
    html.Br(),
    
    # Slider for launch year selection
    dcc.RangeSlider(
        id="year-slider",
        min=df["launch_year"].min(),
        max=df["launch_year"].max(),
        step=1,
        value=[df["launch_year"].min(), df["launch_year"].max()],
        marks={str(year): str(year) for year in sorted(df["launch_year"].unique())}
    ),
    
    html.Br(),
    
    # Scatter plot for payload vs. success
    dcc.Graph(id="success-payload-scatter-chart")
])

In [ ]:
# Step 5: Define interactivity callbacks

@app.callback(
    Output("success-pie-chart", "figure"),
    Input("site-dropdown", "value")
)
def update_pie_chart(selected_site):
    if selected_site == "ALL":
        fig = px.pie(df, names="launch_site", values="success_flag",
                     title="Total Success Launches by Site")
    else:
        filtered_df = df[df["launch_site"] == selected_site]
        fig = px.pie(filtered_df, names="success_flag",
                     title=f"Success vs Failure for site: {selected_site}")
    return fig


@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [Input("site-dropdown", "value"),
     Input("year-slider", "value")]
)
def update_scatter(selected_site, selected_years):
    filtered_df = df[
        (df["launch_year"] >= selected_years[0]) &
        (df["launch_year"] <= selected_years[1])
    ]
    if selected_site != "ALL":
        filtered_df = filtered_df[filtered_df["launch_site"] == selected_site]
    
    fig = px.scatter(
    filtered_df,
    x="payload_mass_kg",
    y="success_flag",
    color="booster_version",
    hover_data=['launch_site', 'orbit']
)

    return fig


In [15]:
if __name__ == "__main__":
    app.run(debug=True)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[13], line 31, in update_scatter(
    selected_site='ALL',
    selected_years=[2024, 2024]
)
     28 if selected_site != "ALL":
     29     filtered_df = filtered_df[filtered_df["launch_site"] == selected_site]
---> 31 fig = px.scatter(
        px = <module 'plotly.express' from 'c:\\Users\\Munawar\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\plotly\\express\\__init__.py'>
        filtered_df =      flight_number                 date booster_version  \
0            286.0  2024-01-03 03:44:00   F9 B5 B1082‑1   
2            287.0  2024-01-03 23:04:00  F9 B5 B1076‑10   
4            288.0  2024-01-07 22:35:00  F9 B5 B1067‑16   
6            289.0  2024-01-14 08:59:00  F9 B5 B1061‑18   
8            290.0  2024-01-15 01:52:00  F9 B5 B1073‑12   
..             ...                  ...             ...   
262      

## ✅ Summary
- Created an interactive Falcon 9 dashboard using Dash and Plotly.
- Included dropdown filtering, year slider, and two key visualizations.
- Ready for local deployment.